In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import json
import requests
import datetime
import dateutil
pd.set_option('display.max_rows', 200)

In [ ]:
def get_json_data(url, auth_token, max_results=None):
    max_results_per_request = 250
    if max_results:
        num_requests_needed = max_results // max_results_per_request + 1
        results_per_request = max_results // num_requests_needed
        if max_results % num_requests_needed != 0:
            results_per_request += 1
    else:
        results_per_request = max_results_per_request
                
    response_json = get_one_json_page(url, auth_token, results_per_request)
    
    next_page_url = response_json['_metadata']['next']
    results = response_json['results']
    
    if max_results:
        return results
    
    while next_page_url is not None:
        response_json = get_one_json_page(next_page_url, auth_token, results_per_request)
        next_page_url = response_json['_metadata']['next']
        results.extend(response_json['results'])
        if max_results and len(results) > max_results:
            break
    
    if max_results:
        results = results[:max_results]
    return results

def get_one_json_page(url, auth_token, results_limit, page=None):
    headers = {'Authorization': 'Bearer {}'.format(auth_token)}
    params = {'limit': results_limit}
    if page:
        params['page'] = page
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

In [ ]:
auth_token = '65122dde5ab2841a7e11e11ed30dde88ffd3b4c0'

In [ ]:
def get_summary(auth_id):
    trip_url = 'https://api.automatic.com/trip/'
    json_trip_data = get_json_data(trip_url, auth_id)
    trip_df = pd.DataFrame(json_trip_data)
    vehicle_url = 'https://api.automatic.com/vehicle'
    json_vehicle_data = get_json_data(vehicle_url, auth_id)
    vehicle_df = pd.DataFrame(json_vehicle_data)
    summary = {}
    summary['overall_summary'] = get_trip_summary(trip_df)
    summary.update(get_per_vehicle_summary(trip_df, vehicle_df))
    return summary

In [ ]:
def get_trip_summary(trip_df):
    summary = {}
    summary.update(get_speeding_fraction_summary(trip_df))
    summary.update(get_vehicle_event_summary(trip_df))
    return summary

In [ ]:
def get_per_vehicle_summary(trip_df, vehicle_df):
    per_vehicle_summaries = {}
    for vehicle_url, df in trip_df.groupby('vehicle'):
        per_vehicle_summaries[vehicle_url] = get_trip_summary(df)
        per_vehicle_summaries[vehicle_url].update(get_vehicle_info(vehicle_url, vehicle_df))
    summary = {}
    summary['per_vehicle_summaries'] = per_vehicle_summaries
    return summary

In [ ]:
def get_vehicle_info(vehicle_url, vehicle_df):
    summary = {}
    if vehicle_url not in vehicle_df.url.values:
        return summary
    vehicle_row = vehicle_df[vehicle_df.url == vehicle_url].iloc[0]
    summary = vehicle_row[['make', 'model', 'submodel', 'year']].to_dict()
#     summary['make'] = vehicle_row['make']
#     summary['model'] = vehicle_row['model']
#     summary['submodel'] = vehicle_row['submodel']
#     summary['year'] = vehicle_row['year']
    return summary

In [ ]:
def get_speeding_fraction_summary(trip_df):
    summary = {}
    highway_duration = (trip_df.duration_s * trip_df.highway_fraction).sum()
    if highway_duration == 0:
        summary['fraction_highway_time_over_70'] = 0
        summary['fraction_highway_time_over_75'] = 0
        summary['fraction_highway_time_over_80'] = 0
    else:
        summary['fraction_highway_time_over_70'] = (trip_df.duration_over_70_s.sum() / 
                                                    highway_duration)
        summary['fraction_highway_time_over_75'] = (trip_df.duration_over_75_s.sum() / 
                                                    highway_duration)
        summary['fraction_highway_time_over_80'] = (trip_df.duration_over_80_s.sum() / 
                                                    highway_duration)
    return summary

In [ ]:
def get_vehicle_event_summary(trip_df):
    kph_to_mph_conversion_factor = 0.621371
    max_speed = 0
    sum_speed_duration_product = 0
    sum_speeding_duration = 0
    sum_gforce_brake = 0
    count_gforce_brake = 0
    max_gforce_brake = 0
    sum_gforce_accel = 0
    count_gforce_accel = 0
    max_gforce_accel = 0
    summary = {}
    for vehicle_event_list in trip_df.vehicle_events:
        for event in vehicle_event_list:
            if event['type'] == 'hard_brake':
                if event['g_force'] > max_gforce_brake:
                    max_gforce_brake = event['g_force']
                sum_gforce_brake += event['g_force']
                count_gforce_brake += 1
            elif event['type'] == 'hard_accel':
                if event['g_force'] > max_gforce_accel:
                    max_gforce_accel = event['g_force']
                sum_gforce_accel += event['g_force']
                count_gforce_accel += 1               
            elif event['type'] == 'speeding':
                event_start = dateutil.parser.parse(event['started_at'])
                event_end = dateutil.parser.parse(event['ended_at'])
                duration = (event_end - event_start).total_seconds()
                if event['velocity_kph'] > max_speed:
                    max_speed = event['velocity_kph']
                sum_speed_duration_product += event['velocity_kph'] * duration
                sum_speeding_duration += duration
    if sum_speeding_duration == 0:
        summary['max_speed_while_speeding_mph'] = 0
        summary['avg_speed_while_speeding_mph'] = 0
    else:
        summary['max_speed_while_speeding_mph'] = max_speed * kph_to_mph_conversion_factor
        summary['avg_speed_while_speeding_mph'] = (sum_speed_duration_product / 
                                                   sum_speeding_duration *
                                                   kph_to_mph_conversion_factor)
    if count_gforce_brake == 0:
        summary['max_gforce_from_hard_brakes'] = 0
        summary['avg_gforce_from_hard_brakes'] = 0
    else:
        summary['max_gforce_from_hard_brakes'] = max_gforce_brake
        summary['avg_gforce_from_hard_brakes'] = sum_gforce_brake / count_gforce_brake
        
    if count_gforce_accel == 0:
        summary['max_gforce_from_hard_accels'] = 0
        summary['avg_gforce_from_hard_accels'] = 0
    else:
        summary['max_gforce_from_hard_accels'] = max_gforce_accel
        summary['avg_gforce_from_hard_accels'] = sum_gforce_accel / count_gforce_accel
    return summary

In [ ]:
summary = get_summary(auth_token)

In [ ]:
summary['overall_summary']

In [ ]:
summ_df = pd.DataFrame(summary['per_vehicle_summaries'])

In [ ]:
summ_df.transpose()

In [ ]:
summary['per_vehicle_summaries']